# Raw Audio Models Training

Training models on raw audio:
- Model 7: Raw Audio CNN
- Model 8: Context-aware Raw Audio

In [2]:
import sys
from pathlib import Path
import torch
import torch.nn as nn
import json

# Determine project root (parent of notebooks directory)
PROJECT_ROOT = Path.cwd().parent if Path.cwd().name in ['notebooks', 'b-p_first_experiments'] else Path.cwd()
sys.path.insert(0, str(PROJECT_ROOT))

from models.raw_audio.raw_audio_cnn import RawAudioCNN
from models.raw_audio.context_audio import ContextAudioModel
from utils.training_utils import train_model, evaluate_model
from utils.data_loader import load_data, create_dataloaders

if torch.backends.mps.is_available():
    device = torch.device("mps")
    print(f"Using MPS device")
else:
    device = torch.device("cpu")

df, spectrograms_dict, feature_cols, feature_scaler, class_weights_dict = load_data(PROJECT_ROOT)
dataloaders = create_dataloaders(df, spectrograms_dict, feature_cols, feature_scaler, class_weights_dict, batch_size=64)

train_audio_loader = dataloaders['raw_audio']['train']
val_audio_loader = dataloaders['raw_audio']['val']
test_audio_loader = dataloaders['raw_audio']['test']

train_context_loader = dataloaders['context_audio']['train']
val_context_loader = dataloaders['context_audio']['val']
test_context_loader = dataloaders['context_audio']['test']

OUTPUT_DIR = PROJECT_ROOT / 'artifacts' / 'b-p_dl_models'
class_weights = torch.tensor([class_weights_dict.get('0', class_weights_dict.get(0, 1.0)), 
                              class_weights_dict.get('1', class_weights_dict.get(1, 1.0))], dtype=torch.float32).to(device)

Using MPS device
Columns in df_phonemes: ['phoneme_id', 'utterance_id', 'phoneme', 'class', 'start_ms', 'end_ms', 'duration_ms', 'audio_path']
Columns in df_features: ['energy_rms', 'energy_rms_std', 'energy_zcr', 'energy_zcr_std', 'spectral_centroid', 'spectral_centroid_std', 'spectral_rolloff', 'spectral_rolloff_std', 'spectral_bandwidth', 'spectral_bandwidth_std', 'formant_f1', 'formant_f2', 'formant_f3', 'formant_f4', 'formant_f1_std', 'formant_f2_std', 'formant_f3_std', 'formant_f4_std', 'spectral_flatness', 'harmonic_noise_ratio', 'zcr_mean', 'energy_cv', 'phoneme_id', 'class', 'duration_ms', 'mfcc_mean_0', 'mfcc_mean_1', 'mfcc_mean_2', 'mfcc_mean_3', 'mfcc_mean_4', 'mfcc_mean_5', 'mfcc_mean_6', 'mfcc_mean_7', 'mfcc_mean_8', 'mfcc_mean_9', 'mfcc_mean_10', 'mfcc_mean_11', 'mfcc_mean_12', 'mfcc_std_0', 'mfcc_std_1', 'mfcc_std_2', 'mfcc_std_3', 'mfcc_std_4', 'mfcc_std_5', 'mfcc_std_6', 'mfcc_std_7', 'mfcc_std_8', 'mfcc_std_9', 'mfcc_std_10', 'mfcc_std_11', 'mfcc_std_12', 'delta_mfcc

## Model 7: Raw Audio CNN

In [2]:
model7 = RawAudioCNN(input_length=3200, num_classes=2).to(device)
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = torch.optim.Adam(model7.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)

save_dir = OUTPUT_DIR / 'raw_audio_models' / 'raw_audio_cnn'
save_dir.mkdir(parents=True, exist_ok=True)

history7, best_epoch7 = train_model(model7, train_audio_loader, val_audio_loader, criterion, optimizer, scheduler,
                                    device, num_epochs=50, save_dir=save_dir, model_name='raw_audio_cnn', early_stopping_patience=10)

checkpoint = torch.load(save_dir / 'best_model.pt')
model7.load_state_dict(checkpoint['model_state_dict'])
test_metrics7, _, _, _ = evaluate_model(model7, test_audio_loader, criterion, device)

with open(save_dir / 'test_metrics.json', 'w') as f:
    json.dump(test_metrics7, f, indent=2)

print(f"Model 7 Test - Acc: {test_metrics7['accuracy']:.4f}, F1: {test_metrics7['f1']:.4f}, ROC-AUC: {test_metrics7['roc_auc']:.4f}")


Epoch 1/50
--------------------------------------------------


Train Loss: 0.2816, Train Acc: 0.8620
Val Loss: 0.5799, Val Acc: 0.6334
Val F1: 0.6379, Val ROC-AUC: 0.9566
✓ New best model saved! (F1: 0.6379)

Epoch 2/50
--------------------------------------------------


Train Loss: 0.2347, Train Acc: 0.8879
Val Loss: 1.2703, Val Acc: 0.3149
Val F1: 0.1678, Val ROC-AUC: 0.9278

Epoch 3/50
--------------------------------------------------


Train Loss: 0.2049, Train Acc: 0.9059
Val Loss: 2.9311, Val Acc: 0.3012
Val F1: 0.1395, Val ROC-AUC: 0.9362

Epoch 4/50
--------------------------------------------------


Train Loss: 0.1902, Train Acc: 0.9110
Val Loss: 1.4208, Val Acc: 0.3485
Val F1: 0.2340, Val ROC-AUC: 0.8896

Epoch 5/50
--------------------------------------------------


...

Train Loss: 0.0393, Train Acc: 0.9840
Val Loss: 0.6367, Val Acc: 0.7381
Val F1: 0.7482, Val ROC-AUC: 0.9534

Epoch 18/50
--------------------------------------------------


Train Loss: 0.0352, Train Acc: 0.9847
Val Loss: 0.3885, Val Acc: 0.9303
Val F1: 0.9296, Val ROC-AUC: 0.9760

Epoch 19/50
--------------------------------------------------


Train Loss: 0.0331, Train Acc: 0.9864
Val Loss: 0.3832, Val Acc: 0.8836
Val F1: 0.8869, Val ROC-AUC: 0.9629

Epoch 20/50
--------------------------------------------------


Train Loss: 0.0285, Train Acc: 0.9878
Val Loss: 0.3224, Val Acc: 0.9027
Val F1: 0.9048, Val ROC-AUC: 0.9707

Early stopping at epoch 20
Best F1: 0.9297 at epoch 10


Model 7 Test - Acc: 0.9275, F1: 0.9271, ROC-AUC: 0.9742


## Model 8: Context-aware Raw Audio

In [3]:
model8 = ContextAudioModel(audio_length=3200, num_classes=2).to(device)
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = torch.optim.Adam(model8.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)

save_dir = OUTPUT_DIR / 'raw_audio_models' / 'context_audio'
save_dir.mkdir(parents=True, exist_ok=True)

history8, best_epoch8 = train_model(model8, train_context_loader, val_context_loader, criterion, optimizer, scheduler,
                                    device, num_epochs=50, save_dir=save_dir, model_name='context_audio', early_stopping_patience=10)

checkpoint = torch.load(save_dir / 'best_model.pt')
model8.load_state_dict(checkpoint['model_state_dict'])
test_metrics8, _, _, _ = evaluate_model(model8, test_context_loader, criterion, device)

with open(save_dir / 'test_metrics.json', 'w') as f:
    json.dump(test_metrics8, f, indent=2)

print(f"Model 8 Test - Acc: {test_metrics8['accuracy']:.4f}, F1: {test_metrics8['f1']:.4f}, ROC-AUC: {test_metrics8['roc_auc']:.4f}")


Epoch 1/50
--------------------------------------------------


Train Loss: 0.3089, Train Acc: 0.8461
Val Loss: 0.3742, Val Acc: 0.8090
Val F1: 0.8168, Val ROC-AUC: 0.9516
✓ New best model saved! (F1: 0.8168)

Epoch 2/50
--------------------------------------------------


Train Loss: 0.2529, Train Acc: 0.8753
Val Loss: 0.9124, Val Acc: 0.4334
Val F1: 0.3780, Val ROC-AUC: 0.9343

Epoch 3/50
--------------------------------------------------


Train Loss: 0.2342, Train Acc: 0.8843
Val Loss: 0.6198, Val Acc: 0.5858
Val F1: 0.5838, Val ROC-AUC: 0.9499

Epoch 4/50
--------------------------------------------------


Train Loss: 0.2234, Train Acc: 0.8910
Val Loss: 0.9842, Val Acc: 0.3291
Val F1: 0.1962, Val ROC-AUC: 0.9484

Epoch 5/50
--------------------------------------------------


...

Train Loss: 0.0536, Train Acc: 0.9760
Val Loss: 0.3008, Val Acc: 0.9273
Val F1: 0.9277, Val ROC-AUC: 0.9729

Epoch 48/50
--------------------------------------------------


Train Loss: 0.0528, Train Acc: 0.9762
Val Loss: 0.2656, Val Acc: 0.9228
Val F1: 0.9240, Val ROC-AUC: 0.9748

Epoch 49/50
--------------------------------------------------


Train Loss: 0.0539, Train Acc: 0.9745
Val Loss: 0.2679, Val Acc: 0.9222
Val F1: 0.9234, Val ROC-AUC: 0.9738

Epoch 50/50
--------------------------------------------------


Train Loss: 0.0539, Train Acc: 0.9748
Val Loss: 0.2989, Val Acc: 0.9286
Val F1: 0.9291, Val ROC-AUC: 0.9751


Model 8 Test - Acc: 0.9269, F1: 0.9268, ROC-AUC: 0.9744
